# Feature reduction
We will sum up the findings from the feature distribution and feature reduction analysis by combining features and selecting which features to use.
We go through each data source: APS-weather, regobs and varsom-forecasts. From there, we built a base of features, before we look into if it is necessary to vary the amount and combination of features for certain models.

The most important labels for the avalanche_ml project are the components of the first avalanche problem wit the following priority.

- dsize (destructive_size_ext_id): size 1-5
- sensitivity to triggers: is a combination of prob and trig e.g., "easy to trigger"
	- prob (aval_probability_id): mulig/sannsynlig
	- trig (aval_trigger_simple_id): stor/liten tilleggsbelastning/naturlig
- dist (aval_distribution_id): få/noen/mange
- problem: new-loose/wet-loose/drift-slab/...
- [lev_min, lev_max] (exposed_height_1/2):
- aspect_* (valid_expositions):

From that we can derive the danger level and emergency warning. The main message, travel advice, and large parts of the bulletin text are probably very correlated to the main avalanche problem, too.

Next on the list would be to know if and what type avalanche problem 2 is.
Here we can just repeat the above list for the main problem.

## APS-weather

- wind_speed_# and wind_change_speed_# should be combined into a new feature wind_speed_max_# = max(wind_speed_#, wind_change_speed_#) and then dropped
- wind_speed_max_0, wind_speed_max_1, wind_speed_max_2 should be summed to wind_speed_max_72h and then dropped

- wind_dir_# we can drop for now

- precip_0, precip_1, precip_2 should be summed to new feature precip_72h and precip_1 and _2 should be dropped
- (we cannot do that for precip_most_exposed since it might not be valid for the same area)

- temp_freeze_#: can we combine these? Aron's approach to take min, max and average over a given time did not seem to improve the performance.

## Regobs

We should consider avalanche activity and here either very recent (_2) activity of all sizes or the release of size-3+ avalanches within the last (2-4 days). The tricky part is how to treat not-observed avalanche activity.

Create a feature regobs_problem_1_dsize_#

- regobs_problem_dsize




In [4]:
from avaml.aggregatedata import ForecastDataset, LabeledData, REG_ENG, CsvMissingError

In [5]:
model_prefix = ''
days = 7
regobs_types = list(REG_ENG.keys())
labeled_data = None
try:
    print("Reading csv")
    labeled_data = LabeledData.from_csv(days=days, regobs_types=regobs_types, with_varsom=False)
except CsvMissingError:
    print("Csv missing. Fetching online data. (This takes a long time.)")
    labeled_data = ForecastDataset(regobs_types=regobs_types).label(days=days, with_varsom=False)
    labeled_data.to_csv()


Reading csv
Csv missing. Fetching online data. (This takes a long time.)
